In [ ]:
!pip install nagisa emoji

In [ ]:
%cd '/content/drive/MyDrive/grad_comp/data'

/content/drive/MyDrive/grad_comp/data


In [ ]:
import nagisa
import unicodedata
import re

KAOMOJI_LEN = 5

def extract_kaomoji(text):
    """ 与えられたテキストから抽出した顔文字リストを返却する。
        → ＼(^o^)／, m(_ _)m などの 手を含む顔文字があれば、それも抽出する
    """
    results = nagisa.extract(text, extract_postags=['補助記号'])
    words = results.words
    kaomoji_words = []
    kaomoji_idx = [i for i, w in enumerate(words) if len(w) >= KAOMOJI_LEN]
    kaomoji_hands = ['ノ', 'ヽ', '∑', 'm', 'O', 'o', '┐', '/', '\\', '┌'] 
    # 顔文字と手を検索
    for i in kaomoji_idx:
        kaomoji = words[i] # 顔文字列
        try:
            # 顔文字の左手
            if words[i-1] in kaomoji_hands and 0 < i:
                kaomoji = words[i-1] + kaomoji
            # 顔文字の右手
            if words[i+1] in kaomoji_hands:
                 kaomoji = kaomoji + words[i+1]
        except IndexError:
            pass
        finally:
            kaomoji_words.append(kaomoji)
    return kaomoji_words

def delete(text, target_list):
    for trg in target_list:
        text = text.replace(trg, "")
    return text

# 顔文字除去したテキストを返す関数
def remove_kaomoji(text):
  res = nagisa.extract(text, extract_postags=['補助記号'])
  kaomoji = res.words # 顔文字リスト
  kaomoji = [t for t in kaomoji if t not in ['、','。','...','?', '?', '!', '!']] # 対象外
  words = nagisa.tagging(text)
  tokens = words.words
  remove_list = [t for t in tokens if t not in kaomoji]
  remove_text = ''.join(remove_list)
  return remove_text

text = "今日は渋谷スクランブルスクエアに行ってきた＼(^o^)／ 夜景🏙サイコー❗️ https://hogehogehogehoge.jpg"
text = unicodedata.normalize('NFKC', text) # NFKC正規化
print(extract_kaomoji(text))
# => ['\\(^o^)/']

In [ ]:
# exp
text = '肉まん、サーモンロール、茶碗蒸？し?、納!豆、シナモンロール！暇やからって食べ過ぎた…(●´з｀●)'
text = unicodedata.normalize('NFKC', text)
res = nagisa.extract(text, extract_postags=['補助記号'])
kaomoji = res.words # 顔文字リスト
words = nagisa.tagging(text)
tokens = words.words
kaomoji = [t for t in kaomoji if t not in ['、','。','...','?', '?', '!', '!']]
remove_list = [t for t in tokens if t not in kaomoji]
remove_text = ''.join(remove_list)
remove_text

'肉まん、サーモンロール、茶碗蒸?し?、納!豆、シナモンロール!暇やからって食べ過ぎた...'

In [ ]:
# ファイル処理
filename = ['train','dev','test']

for name in filename:
  with open('text.' + name + '.txt') as fr,open('../preprocess/text.nagisa_' + name + '.txt',mode='w') as fw:
    for line in fr:
      line = re.sub(r'\s+', '', line) # 空白削除
      line = unicodedata.normalize('NFKC', line) # NFKC正規化
      re_line = remove_kaomoji(line) # 顔文字抽出
      fw.write(re_line + '\n')